In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import xlsxwriter
import os
import yfinance as yf

#### Equal Weights Value Strategy

In [3]:
tickers = pd.read_csv('constituents_csv.csv')
tickers = tickers['Symbol']

valData = pd.DataFrame(columns=['Ticker', 'Price', 'P/E', 'Num Shares to Buy'])

In [4]:
for ticker in tickers:
    curr = yf.Ticker(ticker)
    try:
        temp = curr.info
        valData.loc[len(valData.index)] = [ticker, temp['regularMarketPreviousClose'], temp["forwardPE"], np.nan]
    except:
        print(ticker, " got no data")

valData

ATVI  got no data
ABC  got no data
ANTM  got no data
BLL  got no data
BRK.B  got no data
BF.B  got no data
CERN  got no data
CTXS  got no data
DISCA  got no data
DISCK  got no data
DRE  got no data
RE  got no data
FB  got no data
FRC  got no data
FISV  got no data
FBHS  got no data
INFO  got no data
KSU  got no data
NWS  got no data
NLSN  got no data
NLOK  got no data
PBCT  got no data
PKI  got no data
SIVB  got no data
TWTR  got no data
VIAC  got no data
WLTW  got no data
XLNX  got no data


,Ticker,Price,P/E,Num Shares to Buy
0,MMM,105.56,10.838940,NaN
1,AOS,81.56,20.381546,NaN
2,ABT,108.48,23.548595,NaN
3,ABBV,152.59,13.883513,NaN
4,ABMD,381.02,60.768738,NaN
...,...,...,...,...
472,YUM,129.98,22.298456,NaN
473,ZBRA,265.31,24.033007,NaN
474,ZBH,119.65,15.183879,NaN
475,ZION,43.62,10.784314,NaN


In [5]:
valData.sort_values('P/E', ascending=False, inplace=True)
valData = valData[valData['P/E'] > 0]
valData = valData[:100]
valData.reset_index(inplace=True)
valData.drop('index', axis = 1, inplace=True)

In [6]:
valData

,Ticker,Price,P/E,Num Shares to Buy
0,ILMN,140.23,162.790700,NaN
1,DLR,134.91,115.487180,NaN
2,UDR,37.28,91.829270,NaN
3,WELL,89.95,74.966660,NaN
4,EQIX,801.60,73.432500,NaN
...,...,...,...,...
95,PSA,297.14,26.696236,NaN
96,LIN,409.13,26.619572,NaN
97,ACN,348.75,26.471247,NaN
98,EXPD,126.67,26.415983,NaN


In [8]:
amount = 1000000 #amount you want to put in this portfolio

position_size = float(amount/len(valData.index))
for row in valData.index:
    valData.loc[row, 'Num Shares to Buy'] = position_size // valData.loc[row, 'Price']

valData

,Ticker,Price,P/E,Num Shares to Buy
0,ILMN,140.23,162.790700,71.0
1,DLR,134.91,115.487180,74.0
2,UDR,37.28,91.829270,268.0
3,WELL,89.95,74.966660,111.0
4,EQIX,801.60,73.432500,12.0
...,...,...,...,...
95,PSA,297.14,26.696236,33.0
96,LIN,409.13,26.619572,24.0
97,ACN,348.75,26.471247,28.0
98,EXPD,126.67,26.415983,78.0


#### Advanced Value Strategy

In [5]:
aapl = yf.Ticker('aapl')
aapl.info
for k in list(aapl.info.keys()):
    print(k)

address1
city
state
zip
country
phone
website
industry
industryKey
industryDisp
sector
sectorKey
sectorDisp
longBusinessSummary
fullTimeEmployees
companyOfficers
auditRisk
boardRisk
compensationRisk
shareHolderRightsRisk
overallRisk
governanceEpochDate
compensationAsOfEpochDate
maxAge
priceHint
previousClose
open
dayLow
dayHigh
regularMarketPreviousClose
regularMarketOpen
regularMarketDayLow
regularMarketDayHigh
dividendRate
dividendYield
exDividendDate
payoutRatio
fiveYearAvgDividendYield
beta
trailingPE
forwardPE
volume
regularMarketVolume
averageVolume
averageVolume10days
averageDailyVolume10Day
bid
ask
bidSize
askSize
marketCap
fiftyTwoWeekLow
fiftyTwoWeekHigh
priceToSalesTrailing12Months
fiftyDayAverage
twoHundredDayAverage
trailingAnnualDividendRate
trailingAnnualDividendYield
currency
enterpriseValue
profitMargins
floatShares
sharesOutstanding
sharesShort
sharesShortPriorMonth
sharesShortPreviousMonthDate
dateShortInterest
sharesPercentSharesOut
heldPercentInsiders
heldPercentIn

In [6]:
tickers = pd.read_csv('constituents_csv.csv')
tickers = tickers['Symbol']

advValData = pd.DataFrame(
    columns = [
        'Ticker', 
        'Price', 
        'P/E', 
        'PE Percentile', 
        'P/B Ratio', 
        'PB Percentile', 
        'PEG', 
        'PEG Percentile',
        'EV/EBITDA', 
        'EV/EBITDA Percentile', 
        'EV/Gross Profit', 
        'EV/Profit Percentile',
        'Price/Sales', 
        'PS Percentile',
        'RV Score',
        'Num Shares to Buy'
    ])

for ticker in tickers:
    curr = yf.Ticker(ticker)
    try:
        temp = curr.info
        advValData.loc[len(advValData.index)] = [
            ticker, 
            temp['regularMarketPreviousClose'], 
            temp["forwardPE"], 
            np.nan,
            temp['priceToBook'],
            np.nan,
            temp['pegRatio'],
            np.nan,
            float(temp['enterpriseValue'] / temp['ebitda']),
            np.nan,
            float(temp['enterpriseValue'] / temp['grossProfits']),
            np.nan,
            temp['priceToSalesTrailing12Months'],
            np.nan,
            np.nan,
            np.nan]
    except:
        print(ticker, " got no data")
    
advValData

MMM  got no data
ABMD  got no data
ATVI  got no data
MO  got no data
AAL  got no data
AXP  got no data
AMP  got no data
ABC  got no data
ANTM  got no data
AON  got no data
APA  got no data
AZO  got no data
BLL  got no data
BAC  got no data
BBWI  got no data
BRK.B  got no data
BK  got no data
BA  got no data
BKNG  got no data
BF.B  got no data
COF  got no data
CAH  got no data
CERN  got no data
SCHW  got no data
CTAS  got no data
C  got no data
CFG  got no data
CTXS  got no data
CLX  got no data
CL  got no data
CMA  got no data
CAG  got no data
STZ  got no data
DRI  got no data
DFS  got no data
DISCA  got no data
DISCK  got no data
DPZ  got no data
DRE  got no data
DXC  got no data
EL  got no data
ETSY  got no data
RE  got no data
FB  got no data
FDX  got no data
FITB  got no data
FRC  got no data
FISV  got no data
FBHS  got no data
FOX  got no data
GS  got no data
HCA  got no data
HLT  got no data
HPQ  got no data
HBAN  got no data
INFO  got no data
INTU  got no data
SJM  got no data
J

,Ticker,Price,P/E,PE Percentile,P/B Ratio,PB Percentile,PEG,PEG Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Profit Percentile,Price/Sales,PS Percentile,RV Score,Num Shares to Buy
0,AOS,81.56,20.381546,NaN,6.457297,NaN,2.70,NaN,14.822572,NaN,9.008815,NaN,3.191337,NaN,NaN,NaN
1,ABT,108.48,23.548595,NaN,5.050023,NaN,-12.21,NaN,20.073697,NaN,8.061657,NaN,4.736921,NaN,NaN,NaN
2,ABBV,152.59,13.883513,NaN,22.618979,NaN,-3.31,NaN,11.484277,NaN,7.741454,NaN,4.961238,NaN,NaN,NaN
3,ACN,348.75,26.471247,NaN,8.330788,NaN,3.70,NaN,19.372417,NaN,10.562651,NaN,3.440832,NaN,NaN,NaN
4,ADM,70.93,10.993846,NaN,1.515428,NaN,-1.84,NaN,9.133338,NaN,6.138761,NaN,0.393364,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,XEL,61.53,17.335196,NaN,1.977945,NaN,2.91,NaN,12.060448,NaN,10.536693,NaN,2.311244,NaN,NaN,NaN
379,XYL,111.69,27.675674,NaN,2.731461,NaN,1.60,NaN,25.765863,NaN,13.808430,NaN,4.021775,NaN,NaN,NaN
380,ZBRA,265.31,24.033007,NaN,4.592346,NaN,5.93,NaN,17.218230,NaN,6.187801,NaN,2.724850,NaN,NaN,NaN
381,ZBH,119.65,15.183879,NaN,2.014033,NaN,2.30,NaN,12.842248,NaN,6.220676,NaN,3.461197,NaN,NaN,NaN


In [7]:
from scipy import stats

In [8]:
metrics = {
    'P/E': 'PE Percentile',
    'P/B Ratio':'PB Percentile',
    'PEG':'PEG Percentile',
    'Price/Sales': 'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Gross Profit':'EV/Profit Percentile'
}

for row in advValData.index:
    for metric in metrics:
        advValData.loc[row, metrics[metric]] = stats.percentileofscore(advValData[metric], advValData.loc[row, metric])/100
    
for metric in metrics.values():
    print(advValData[metric])

advValData

0      0.590078
1      0.710183
2      0.295039
3      0.775457
4      0.164491
         ...   
378    0.488251
379    0.806789
380    0.725849
381    0.362924
382    0.885117
Name: PE Percentile, Length: 383, dtype: float64
0      0.733681
1      0.647520
2      0.950392
3      0.822454
4      0.122715
         ...   
378    0.224543
379    0.407311
380    0.618799
381    0.237598
382    0.916449
Name: PB Percentile, Length: 383, dtype: float64
0      0.684073
1      0.031332
2      0.073107
3      0.831593
4      0.109661
         ...   
378    0.720627
379    0.420366
380    0.916449
381    0.592689
382    0.788512
Name: PEG Percentile, Length: 383, dtype: float64
0      0.561358
1      0.712794
2      0.741514
3      0.577023
4      0.049608
         ...   
378    0.454308
379    0.663185
380    0.511749
381    0.582245
382    0.932115
Name: PS Percentile, Length: 383, dtype: float64
0      0.488251
1      0.757180
2      0.308094
3      0.728460
4      0.167102
         ...   
378

,Ticker,Price,P/E,PE Percentile,P/B Ratio,PB Percentile,PEG,PEG Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Profit Percentile,Price/Sales,PS Percentile,RV Score,Num Shares to Buy
0,AOS,81.56,20.381546,0.590078,6.457297,0.733681,2.70,0.684073,14.822572,0.488251,9.008815,0.493473,3.191337,0.561358,NaN,NaN
1,ABT,108.48,23.548595,0.710183,5.050023,0.647520,-12.21,0.031332,20.073697,0.757180,8.061657,0.441253,4.736921,0.712794,NaN,NaN
2,ABBV,152.59,13.883513,0.295039,22.618979,0.950392,-3.31,0.073107,11.484277,0.308094,7.741454,0.412533,4.961238,0.741514,NaN,NaN
3,ACN,348.75,26.471247,0.775457,8.330788,0.822454,3.70,0.831593,19.372417,0.728460,10.562651,0.603133,3.440832,0.577023,NaN,NaN
4,ADM,70.93,10.993846,0.164491,1.515428,0.122715,-1.84,0.109661,9.133338,0.167102,6.138761,0.300261,0.393364,0.049608,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,XEL,61.53,17.335196,0.488251,1.977945,0.224543,2.91,0.720627,12.060448,0.344648,10.536693,0.600522,2.311244,0.454308,NaN,NaN
379,XYL,111.69,27.675674,0.806789,2.731461,0.407311,1.60,0.420366,25.765863,0.866841,13.808430,0.770235,4.021775,0.663185,NaN,NaN
380,ZBRA,265.31,24.033007,0.725849,4.592346,0.618799,5.93,0.916449,17.218230,0.621410,6.187801,0.305483,2.724850,0.511749,NaN,NaN
381,ZBH,119.65,15.183879,0.362924,2.014033,0.237598,2.30,0.592689,12.842248,0.396867,6.220676,0.310705,3.461197,0.582245,NaN,NaN


In [9]:
from statistics import mean

for row in advValData.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(advValData.loc[row, metrics[metric]])
    advValData.loc[row, 'RV Score'] = mean(value_percentiles)

advValData

,Ticker,Price,P/E,PE Percentile,P/B Ratio,PB Percentile,PEG,PEG Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Profit Percentile,Price/Sales,PS Percentile,RV Score,Num Shares to Buy
0,AOS,81.56,20.381546,0.590078,6.457297,0.733681,2.70,0.684073,14.822572,0.488251,9.008815,0.493473,3.191337,0.561358,0.591819,NaN
1,ABT,108.48,23.548595,0.710183,5.050023,0.647520,-12.21,0.031332,20.073697,0.757180,8.061657,0.441253,4.736921,0.712794,0.550044,NaN
2,ABBV,152.59,13.883513,0.295039,22.618979,0.950392,-3.31,0.073107,11.484277,0.308094,7.741454,0.412533,4.961238,0.741514,0.463446,NaN
3,ACN,348.75,26.471247,0.775457,8.330788,0.822454,3.70,0.831593,19.372417,0.728460,10.562651,0.603133,3.440832,0.577023,0.723020,NaN
4,ADM,70.93,10.993846,0.164491,1.515428,0.122715,-1.84,0.109661,9.133338,0.167102,6.138761,0.300261,0.393364,0.049608,0.152306,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,XEL,61.53,17.335196,0.488251,1.977945,0.224543,2.91,0.720627,12.060448,0.344648,10.536693,0.600522,2.311244,0.454308,0.472150,NaN
379,XYL,111.69,27.675674,0.806789,2.731461,0.407311,1.60,0.420366,25.765863,0.866841,13.808430,0.770235,4.021775,0.663185,0.655788,NaN
380,ZBRA,265.31,24.033007,0.725849,4.592346,0.618799,5.93,0.916449,17.218230,0.621410,6.187801,0.305483,2.724850,0.511749,0.616623,NaN
381,ZBH,119.65,15.183879,0.362924,2.014033,0.237598,2.30,0.592689,12.842248,0.396867,6.220676,0.310705,3.461197,0.582245,0.413838,NaN


In [10]:
advValData.sort_values(by = "RV Score", inplace = True)
advValData = advValData[:50]
advValData.reset_index(drop = True, inplace = True)

In [11]:
amount = 1000000
advValData

,Ticker,Price,P/E,PE Percentile,P/B Ratio,PB Percentile,PEG,PEG Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Profit Percentile,Price/Sales,PS Percentile,RV Score,Num Shares to Buy
0,LUMN,1.74,-17.300000,0.005222,0.786721,0.013055,-0.74,0.161880,4.535528,0.023499,2.250301,0.031332,0.118294,0.002611,0.039600,NaN
1,UAL,42.58,4.450837,0.018277,1.576510,0.138381,0.07,0.198433,4.159176,0.020888,2.546415,0.046997,0.266081,0.015666,0.073107,NaN
2,MHK,102.95,10.331664,0.120104,0.903752,0.015666,-1.99,0.107050,6.777854,0.088773,3.010155,0.080940,0.587664,0.083551,0.082681,NaN
3,VTRS,10.65,3.803571,0.013055,0.612315,0.005222,-1.44,0.122715,6.021406,0.065274,4.223522,0.167102,0.826123,0.130548,0.083986,NaN
4,VLO,131.92,9.161468,0.083551,1.732748,0.182768,-0.25,0.182768,3.030993,0.015666,2.741359,0.057441,0.309428,0.028721,0.091819,NaN
5,LNC,27.48,3.895565,0.015666,2.087550,0.255875,0.64,0.233681,-1.142518,0.013055,-4.390789,0.007833,0.316240,0.031332,0.092907,NaN
6,MOS,36.33,10.776786,0.146214,0.986622,0.020888,-0.24,0.185379,5.470725,0.044386,2.645637,0.052219,0.790127,0.122715,0.095300,NaN
7,AIG,66.57,8.841479,0.075718,1.193878,0.052219,0.63,0.229765,5.053614,0.039164,2.391804,0.036554,0.936921,0.148825,0.097041,NaN
8,ALK,40.59,8.103586,0.065274,1.263432,0.065274,0.54,0.224543,7.779886,0.125326,2.921171,0.067885,0.503207,0.067885,0.102698,NaN
9,CNC,73.41,10.769456,0.143603,1.550004,0.133159,0.94,0.300261,5.799527,0.057441,1.551913,0.010444,0.281390,0.018277,0.110531,NaN


In [14]:
position_size = float(amount) / len(advValData.index)
for row in advValData.index:
    advValData.loc[row, 'Num Shares to Buy'] = position_size // advValData.loc[row, 'Price']

advValData

,Ticker,Price,P/E,PE Percentile,P/B Ratio,PB Percentile,PEG,PEG Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit,EV/Profit Percentile,Price/Sales,PS Percentile,RV Score,Num Shares to Buy
0,LUMN,1.74,-17.300000,0.005222,0.786721,0.013055,-0.74,0.161880,4.535528,0.023499,2.250301,0.031332,0.118294,0.002611,0.039600,11494.0
1,UAL,42.58,4.450837,0.018277,1.576510,0.138381,0.07,0.198433,4.159176,0.020888,2.546415,0.046997,0.266081,0.015666,0.073107,469.0
2,MHK,102.95,10.331664,0.120104,0.903752,0.015666,-1.99,0.107050,6.777854,0.088773,3.010155,0.080940,0.587664,0.083551,0.082681,194.0
3,VTRS,10.65,3.803571,0.013055,0.612315,0.005222,-1.44,0.122715,6.021406,0.065274,4.223522,0.167102,0.826123,0.130548,0.083986,1877.0
4,VLO,131.92,9.161468,0.083551,1.732748,0.182768,-0.25,0.182768,3.030993,0.015666,2.741359,0.057441,0.309428,0.028721,0.091819,151.0
5,LNC,27.48,3.895565,0.015666,2.087550,0.255875,0.64,0.233681,-1.142518,0.013055,-4.390789,0.007833,0.316240,0.031332,0.092907,727.0
6,MOS,36.33,10.776786,0.146214,0.986622,0.020888,-0.24,0.185379,5.470725,0.044386,2.645637,0.052219,0.790127,0.122715,0.095300,550.0
7,AIG,66.57,8.841479,0.075718,1.193878,0.052219,0.63,0.229765,5.053614,0.039164,2.391804,0.036554,0.936921,0.148825,0.097041,300.0
8,ALK,40.59,8.103586,0.065274,1.263432,0.065274,0.54,0.224543,7.779886,0.125326,2.921171,0.067885,0.503207,0.067885,0.102698,492.0
9,CNC,73.41,10.769456,0.143603,1.550004,0.133159,0.94,0.300261,5.799527,0.057441,1.551913,0.010444,0.281390,0.018277,0.110531,272.0
